<a href="https://colab.research.google.com/github/readylilian/MusicGenerator/blob/master/Rip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
#This is where the pre-trained music vae models are stored 
BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"

print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -qU magenta

print('Importing libraries and defining some helper functions...')
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#Allows a music sequence to be playable and to pop up
#Plays the note sequence
def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

#Creates an average of whatever sequences are fed into it (averaging between the start and end sequence)
def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5,
                individual_duration=4.0):
  """Interpolates between a start and end sequence."""
  note_sequences = model.interpolate(
      start_seq, end_seq,num_steps=num_steps, length=max_length,
      temperature=temperature,
      assert_same_length=assert_same_length)
  ##Combines all the averaged pieces together
  interp_seq = mm.sequences_lib.concatenate_sequences(
      note_sequences, [individual_duration] * len(note_sequences))
  ##Allows you to play the averaged sequence
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]

print('Done')

Installing dependencies...
Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 160983 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 1.4MB 21.5MB/s 
     |████████████████████████████████| 204kB 43.7MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 
     |████████████████████████████████| 1.6MB 38.2MB/s 
     |████████████████████████████████| 3.6MB 36.0MB/s 
     |█████████████████████████

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


Instructions for updating:
non-resource variables are not supported in the long term
Done


In [2]:
#uses the default configs for this model
hierdec_trio_16bar_config = configs.CONFIG_MAP['hierdec-trio_16bar']
#uses the configs and the checkpoint that's been stored to initialize the model
hierdec_trio_16bar = TrainedModel(hierdec_trio_16bar_config, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/trio_16bar_hierdec.ckpt')

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, HierarchicalLstmDecoder, and hparams:
{'max_seq_len': 256, 'z_size': 512, 'free_bits': 256, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [1024, 1024], 'enc_rnn_size': [2048, 2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'constant', 'sampling_rate': 0.0, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048, 2048]

INFO:tensorflow:
Hierarchical Decoder:
  input length: 256
  level output lengths: [16, 16]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]

INFO:tensorflow:
Decoder Cells:
  units: [1024, 1024]



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
INFO:tensorflow:Restoring parameters from gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/trio_16bar_hierdec.ckpt


In [3]:
#Upload whatever midi files you want to combine
input_trio_midi_data = files.upload().values() or input_trio_midi_data

Saving Blink_182_-_What's_My_Age_Again.mid to Blink_182_-_What's_My_Age_Again.mid
Saving Bobby_Fuller_Four_-_I_Fought_The_Law.mid to Bobby_Fuller_Four_-_I_Fought_The_Law.mid
Saving Green_Day_-_Basket_Case.mid to Green_Day_-_Basket_Case.mid
Saving LightEmUp.mid to LightEmUp.mid
Saving Queen_-_Killer_Queen.mid to Queen_-_Killer_Queen.mid
Saving THE BEATLES.I want you shes so heavy.mid to THE BEATLES.I want you shes so heavy.mid
Saving The_Clash_-_Should_I_Stay_or_Should_I_Go.mid to The_Clash_-_Should_I_Stay_or_Should_I_Go.mid
Saving The_Eurythmics_-_Sweet_Dreams.mid to The_Eurythmics_-_Sweet_Dreams.mid
Saving The_Offspring_-_Pretty_Fly.mid to The_Offspring_-_Pretty_Fly.mid
Saving The_Ramones_-_I_Wanna_Be_Sedated.mid to The_Ramones_-_I_Wanna_Be_Sedated.mid


In [4]:
#Makes all uploaded midi files to note sequences (from music to math representation)
trio_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_trio_midi_data]
extracted_trios = []
#Convert all notesequences to ones that can be read through the trio model (throws out unusable notes)
for ns in trio_input_seqs:
  extracted_trios.extend(
      hierdec_trio_16bar_config.data_converter.from_tensors(
          hierdec_trio_16bar_config.data_converter.to_tensors(ns)[1]))

/usr/local/lib/python3.7/dist-packages/pretty_midi/pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


In [5]:
interpolated_seq = []
#Work through each song in the trio and combine them
def iterate_seq():
  j = 0
  i = 1

  while i < len(extracted_trios):
    start_trio = extracted_trios[j]
    end_trio = extracted_trios[i]
    trio_16bar_mean = interpolate(hierdec_trio_16bar, start_trio, end_trio, num_steps=3, max_length=256, individual_duration=32, temperature=0.5)
    interpolated_seq.append(trio_16bar_mean)
    j+=2
    i+=2



In [ ]:
# While there are still songs to combine, continue to iterate
#Copy and clear to keep using the same variables
while len(interpolated_seq) != 1:
  iterate_seq()
  if len(interpolated_seq) == 1 or len(interpolated_seq) == 0:
    #If one song is left stop iterating
    print("Done!")
    break
  extracted_trios.clear
  extracted_trios = interpolated_seq.copy()
  print(len(extracted_trios))
  interpolated_seq.clear()